# II. Simulate energy injection of your favorite model

`DM21cm` can take any energy injection of the form
$$\frac{\mathrm{d}N^{\gamma,e^\pm}}{\mathrm{d}E}(z, E)\cdot\Gamma(z, \delta(\vec x), T_k(\vec x), x_e(\vec x), \vec x)$$
i.e. redshift-dependent $\gamma$ and $e^\pm$ spectra, times a redshift & location-dependent injections rate $\Gamma$.

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
from astropy.cosmology import Planck18
import py21cmfast as p21c

WDIR = os.environ['DM21CM_DIR']
sys.path.append(WDIR)
from dm21cm.evolve import evolve
from dm21cm.utils import abscs

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:59: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:43: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


Default cache dir: /n/netscratch/iaifi_lab/Lab/yitians/dm21cm/21cmFAST-cache


## 1. Example custom injection model
We take a look at the `Injection` base class, and see what we should change.

In [ ]:
from dm21cm.injections.base import Injection
from darkhistory.spec import pppc

In [ ]:
class DMDecayInjection (Injection):
    """Dark matter decay injection object. See parent class for details.
    
    Args:
        primary (str): Primary injection channel. See darkhistory.pppc.get_pppc_spec
        m_DM (float): DM mass in [eV].
        lifetime (float, optional): Decay lifetime in [s].
    """

    def __init__(self, primary=None, m_DM=None, lifetime=None):
        self.mode = 'DM decay'
        self.primary = primary
        self.m_DM = m_DM
        self.lifetime = lifetime

        self.phot_spec_per_inj = pppc.get_pppc_spec(
            self.m_DM, abscs['photE'], self.primary, 'phot', decay=True
        ) # [phot / inj]
        self.elec_spec_per_inj = pppc.get_pppc_spec(
            self.m_DM, abscs['elecEk'], self.primary, 'elec', decay=True
        ) # [elec / inj]

    def is_injecting_elec(self):
        return not np.allclose(self.elec_spec_per_inj.N, 0.)
    
    def get_config(self):
        return {
            'mode': self.mode,
            'primary': self.primary,
            'm_DM': self.m_DM,
            'lifetime': self.lifetime
        }

    #===== injections =====
    # Assuming Euler steps. z_end is not used.
    def inj_rate(self, z, z_end=None, **kwargs):
        rho_DM = phys.rho_DM * (1+z)**3 # [eV / pcm^3]
        return float((rho_DM/self.m_DM) / self.lifetime) # [inj / pcm^3 s]
    
    def inj_power(self, z, z_end=None, **kwargs):
        return self.inj_rate(z) * self.m_DM # [eV / pcm^3 s]
    
    def inj_phot_spec(self, z, z_end=None, **kwargs):
        return self.phot_spec_per_inj * self.inj_rate(z) # [phot / pcm^3 s]
    
    def inj_elec_spec(self, z, z_end=None, **kwargs):
        return self.elec_spec_per_inj * self.inj_rate(z) # [elec / pcm^3 s]
    
    def inj_phot_spec_box(self, z, z_end=None, delta_plus_one_box=None, **kwargs):
        return self.inj_phot_spec(z), delta_plus_one_box # [phot / pcm^3 s], [1]

    def inj_elec_spec_box(self, z, z_end=None, delta_plus_one_box=None, **kwargs):
        return self.inj_elec_spec(z), delta_plus_one_box # [elec / pcm^3 s], [1]

In [ ]:
Injection?

We just need to specify the following basic functions
- `__init__`
- `is_injecting_elec`
- `get_config`

and the injection functions

- `inj_rate`
- `inj_power`
- `inj_phot_spec`
- `inj_elec_spec`
- `inj_phot_spec_box`
- `inj_elec_spec_box`.

To demonstrate, we look at the docstrings of the base class along with implementation of the DM decay injection.

### 1.1 First, some helper methods

In [4]:
abstract_injection = Injection()

In [ ]:
# Initialization of DMDecayInjection
def __init__(self, primary=None, m_DM=None, lifetime=None):
    self.mode = 'DM decay'
    self.primary = primary
    self.m_DM = m_DM
    self.lifetime = lifetime

    self.phot_spec_per_inj = pppc.get_pppc_spec(
        self.m_DM, abscs['photE'], self.primary, 'phot', decay=True
    ) # [phot / inj]
    self.elec_spec_per_inj = pppc.get_pppc_spec(
        self.m_DM, abscs['elecEk'], self.primary, 'elec', decay=True
    ) # [elec / inj]

In [5]:
abstract_injection.is_injecting_elec?

Signature: abstract_injection.is_injecting_elec()
Docstring:
Whether DM is injecting electron/positron. Used by evolve.

Returns:
    bool: Whether DM is injecting electron/positron.
File:      ~/dm21cm/DM21cm/dm21cm/injections/base.py
Type:      method

In [ ]:
# Implementation in DMDecayInjection
def is_injecting_elec(self):
    return not np.allclose(self.elec_spec_per_inj.N, 0.)

In [6]:
abstract_injection.get_config?

Signature: abstract_injection.get_config()
Docstring:
Get configuration of the injection.
Used in DM21cm's DarkHistory wrapper to check if cached solution has the correct injection.

Returns:
    dict: Configuration of the injection.
File:      ~/dm21cm/DM21cm/dm21cm/injections/base.py
Type:      method

In [ ]:
# Implementation in DMDecayInjection
def get_config(self):
    return {
        'mode': self.mode,
        'primary': self.primary,
        'm_DM': self.m_DM,
        'lifetime': self.lifetime
    }

### 1.2 Injection functions

#### 1.2.1 Overall parameters

In [7]:
abstract_injection.inj_rate?
#abstract_injection.inj_power?

Signature: abstract_injection.inj_rate(z_start, z_end=None, state=None)
Docstring:
Injection event rate density in [inj / pcm^3 s].
Used in DarkHistory. Assumes a homogeneous universe.
If injection cannot be thought of as events, use 1 injection per second by convention.
This factor is kept for DarkHistory's API, but will cancel out in the final result.

Args:
    z_start (float): Starting redshift of the redshift step of injection.
    z_end (float, optional): Ending redshift of the redshift step of injection.
        Useful for calculating the average rate over a redshift step when injection rate evolves quickly.
        If None, must return instantaneous rate. This is used in DarkHistory's TLA integrator.
        For slow varying rates, it is sufficient to return the rate at the starting redshift for Euler steppping.
    state (dict, optional): State of the universe at z_start. Used for rates with feedback.

Returns:
    float: Injection event rate per average baryon in [inj / Bavg 

In [ ]:
# Implementation in DMDecayInjection
def inj_rate(self, z):
    rho_DM = phys.rho_DM * (1+z)**3 # [eV / pcm^3]
    return float((rho_DM/self.m_DM) / self.lifetime) # [inj / pcm^3 s]

def inj_power(self, z):
    return self.inj_rate(z) * self.m_DM # [eV / pcm^3 s]

#### 1.2.2 Injection into a homogeneous universe (for DarkHistory)

In [8]:
abstract_injection.inj_phot_spec?
#abstract_injection.inj_elec_spec?

Signature: abstract_injection.inj_phot_spec(z_start, z_end=None, state=None, **kwargs)
Docstring:
Injected photon rate density spectrum assuming a homogeneous universe.
Used in DarkHistory.

Args:
    z_start (float): Starting redshift of the redshift step of injection.
    z_end (float, optional): Ending redshift of the redshift step of injection. See details in inj_rate.
    state (dict, optional): State of the universe at z_start. Used for rates with feedback.

Returns:
    Spectrum: Injected photon rate spectrum in [phot / pcm^3 s].
        Spectrum value 'spec' can be either 'N' (particle in bin) or 'dNdE'.
        See darkhistory.spec.spectrum.Spectrum
File:      ~/dm21cm/DM21cm/dm21cm/injections/base.py
Type:      method

In [ ]:
# Implementation in DMDecayInjection
def inj_phot_spec(self, z, **kwargs):
    return self.phot_spec_per_inj * self.inj_rate(z) # [phot / pcm^3 s]

def inj_elec_spec(self, z, **kwargs):
    return self.elec_spec_per_inj * self.inj_rate(z) # [elec / pcm^3 s]

#### 1.2.3 Injection in DM21cm inhomogeneously

In [9]:
abstract_injection.inj_phot_spec_box?
#abstract_injection.inj_elec_spec_box?

Signature:
abstract_injection.inj_phot_spec_box(
    z_start,
    z_end=None,
    state=None,
    **kwargs,
)
Docstring:
Injected photon rate density spectrum and weight box.
Called in dm21cm.evolve every redshift step.

Args:
    z_start (float): Starting redshift of the redshift step of injection.
    z_end (float, optional): Ending redshift of the redshift step of injection. See details in inj_rate.
    state (dict, optional): State of the universe at z_start. Used for rates with feedback.

Returns:
    tuple : (spec, weight_box), where:
        spec (Spectrum) : Injected photon rate density spectrum [spec / pcm^3 s].
        weight_box (ndarray) : Injection weight box of the above spectrum [1].

Note:
    The output injection is spec \otimes weight_box, with spec carrying the units.
File:      ~/dm21cm/DM21cm/dm21cm/injections/base.py
Type:      method

In [ ]:
# Implementation in DMDecayInjection
def inj_phot_spec_box(self, z, delta_plus_one_box=None, **kwargs):
    return self.inj_phot_spec(z), delta_plus_one_box # [phot / pcm^3 s], [1]

def inj_elec_spec_box(self, z, delta_plus_one_box=None, **kwargs):
    return self.inj_elec_spec(z), delta_plus_one_box # [elec / pcm^3 s], [1]

## 2. After implementing the above methods, pass it into `evolve`

In [ ]:
return_dict = evolve(
    run_name = 'test',
    z_start = 45.,
    z_end = 5.,
    subcycle_factor = 10,
    dm_params = DMDecayInjection(
        primary='phot_delta',
        m_DM=1e8, # [eV]
        lifetime=1e28, # [s]
    ),
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 64,
            BOX_LEN = 256, # [conformal Mpc]
            N_THREADS = 32,
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = Planck18.Om0,
            OMb = Planck18.Ob0,
            POWER_INDEX = Planck18.meta['n'],
            SIGMA_8 = Planck18.meta['sigma8'],
            hlittle = Planck18.h,
        ),
        random_seed = 12345,
        write = True,
    ),
    p21c_astro_params = p21c.AstroParams(),
)